In [4]:
!pip install cupy

     ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
     --- ------------------------------------ 0.3/3.4 MB ? eta -:--:--
     ------------ --------------------------- 1.0/3.4 MB 3.6 MB/s eta 0:00:01
     ------------------------ --------------- 2.1/3.4 MB 4.5 MB/s eta 0:00:01
     ------------------------------ --------- 2.6/3.4 MB 4.4 MB/s eta 0:00:01
     ---------------------------------- ----- 2.9/3.4 MB 4.1 MB/s eta 0:00:01
     ---------------------------------------- 3.4/3.4 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached fastrlock-0.8.2-cp310-cp310-win_amd64.whl.metadata (9.6 kB)
Using cached fastrlock-0.8.2-cp310-cp310-win_amd64.whl (29 kB)
  Running setup.py clean for cupy
Failed to build cupy


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [57 lines of output]
  Generating cache key from header files...
  Cache key (1610 files matching C:\Users\nihal\AppData\Local\Temp\pip-install-hxjp550h\cupy_b450889ecb0d41d487c8e610f93e18bb\cupy\_core\include\**): 6d15f8ff018dac193d6bd2dcc599f463a0e286a8
  Clearing directory: C:\Users\nihal\AppData\Local\Temp\pip-install-hxjp550h\cupy_b450889ecb0d41d487c8e610f93e18bb\cupy\.data
  Looking for NVTX: C:\Program Files\NVIDIA Corporation\Nsight Systems *\target-windows-x64\nvtx
  NVTX could not be found
  
  -------- Configuring Module: cuda --------
  Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  **************************************************
  *** WARNING: Cannot check compute capability
  Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++

In [3]:
!pip install numba

In [5]:
import os
import numpy as np
import cupy as cp
from scipy.signal import butter, filtfilt
import networkx as nx
from scipy.ndimage import zoom
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
from numba import jit, prange

# Load and Validate Data
def load_and_validate_data(file_path):
    try:
        loaded_data = np.load(file_path)
        ppg_f = loaded_data.get('ppg_f')
        ecg_f = loaded_data.get('ecg_f')
        seg_dbp = loaded_data.get('seg_dbp')
        seg_sbp = loaded_data.get('seg_sbp')
        
        if ppg_f is None or ecg_f is None or seg_dbp is None or seg_sbp is None:
            return None
        
        if ppg_f.size == 0 or ecg_f.size == 0 or seg_dbp.size == 0 or seg_sbp.size == 0:
            return None

        return ppg_f, ecg_f, seg_dbp, seg_sbp
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# Bandpass Filter using GPU (CuPy)
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    
    data_gpu = cp.asarray(data)
    filtered_data_gpu = filtfilt(b, a, data_gpu)
    return cp.asnumpy(filtered_data_gpu)

# Preprocess Signal using GPU (CuPy)
def preprocess_signal(signal, fs):
    filtered_signal = bandpass_filter(signal, 0.5, 40, fs)
    normalized_signal = (filtered_signal - cp.mean(filtered_signal)) / cp.std(filtered_signal)
    return cp.asnumpy(normalized_signal)

# Visibility Graph Creation (Numba)
@jit(nopython=True, parallel=True)
def create_visibility_graph(ppg_signal):
    n = len(ppg_signal)
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for i in prange(n):
        for j in range(i + 1, n):
            visible = True
            for k in range(i + 1, j):
                if ppg_signal[k] >= ppg_signal[i] + (ppg_signal[j] - ppg_signal[i]) * (k - i) / (j - i):
                    visible = False
                    break
            if visible:
                G.add_edge(i, j)
                
    return G

# Convert Graph to Adjacency Matrix Image (CuPy)
def graph_to_adjacency_matrix_image(G, size):
    adj_matrix = cp.asarray(nx.to_numpy_array(G))
    adj_matrix_resized = zoom(adj_matrix, (size / adj_matrix.shape[0], size / adj_matrix.shape[1]), order=0)
    return adj_matrix_resized

def graph_to_adjacency_matrix_GCN(G):
    return nx.adjacency_matrix(G).todense()

# Generate VG Image (CuPy)
def generate_vg_image(ppg_signal, size):
    G = create_visibility_graph(ppg_signal)
    vg_image = graph_to_adjacency_matrix_image(G, size)
    return cp.asnumpy(vg_image)

# Process Signal and Generate VG Image
def process_signal(i, ppg_signal, vg_image_size):
    print(f"Generating VG image for PPG signal {i + 1}")
    ppg_signal = ppg_signal.flatten()
    vg_image = generate_vg_image(ppg_signal, vg_image_size)
    return vg_image.flatten()

# Combine and Process Data from Folder
def combine_data_from_folder(folder_path, batch_size=500):
    combined_ppg = []
    combined_ecg = []
    combined_seg_dbp = []
    combined_seg_sbp = []

    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        if not file_path.endswith('.npz'):
            continue
        
        data = load_and_validate_data(file_path)
        
        if data is None:
            print(f"Skipping invalid file: {file_path}")
            continue
        
        ppg_f, ecg_f, seg_dbp, seg_sbp = data
        
        if ppg_f.ndim == 3:
            for i in range(ppg_f.shape[0]):
                preprocessed_ppg = preprocess_signal(ppg_f[i], 1000)
                combined_ppg.append(preprocessed_ppg)
        else:
            print(f"Skipping PPG data with unexpected shape: {ppg_f.shape}")
        
        if ecg_f.ndim == 3:
            for i in range(ecg_f.shape[0]):
                preprocessed_ecg = preprocess_signal(ecg_f[i], 1000)
                combined_ecg.append(preprocessed_ecg)
        else:
            print(f"Skipping ECG data with unexpected shape: {ecg_f.shape}")
        
        if seg_dbp.ndim == 2:
            combined_seg_dbp.append(seg_dbp)
        else:
            print(f"Skipping SegDBP data with unexpected shape: {seg_dbp.shape}")
        
        if seg_sbp.ndim == 2:
            combined_seg_sbp.append(seg_sbp)
        else:
            print(f"Skipping SegSBP data with unexpected shape: {seg_sbp.shape}")
        
        if len(combined_ppg) >= batch_size:
            combined_ppg = np.stack(combined_ppg, axis=0)
            combined_ecg = np.stack(combined_ecg, axis=0)
            combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
            combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
            
            yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp
            
            combined_ppg = []
            combined_ecg = []
            combined_seg_dbp = []
            combined_seg_sbp = []

    if combined_ppg:
        combined_ppg = np.stack(combined_ppg, axis=0)
    else:
        combined_ppg = np.array([])
        
    if combined_ecg:
        combined_ecg = np.stack(combined_ecg, axis=0)
    else:
        combined_ecg = np.array([])
        
    if combined_seg_dbp:
        combined_seg_dbp = np.concatenate(combined_seg_dbp, axis=0)
    else:
        combined_seg_dbp = np.array([])
        
    if combined_seg_sbp:
        combined_seg_sbp = np.concatenate(combined_seg_sbp, axis=0)
    else:
        combined_seg_sbp = np.array([])

    yield combined_ppg, combined_ecg, combined_seg_dbp, combined_seg_sbp

# Invert VG Images
def invert_images(vg_images):
    inverted_images = [255 - image for image in vg_images]
    return inverted_images

# Directory and Image Size
folder_path = r'C:\Users\nihal\Desktop\NIHAL_IMP_DOCS\Internship_PPG\Validation_data'
vg_image_size = 224  # VG image size

# Create Output Directory
output_dir = 'vg_images_validation'
os.makedirs(output_dir, exist_ok=True)

# Data Generator for Validation Data
val_data_generator = combine_data_from_folder(folder_path, 500)

ModuleNotFoundError: No module named 'cupy'

In [ ]:
# Process and Save VG Images
for batch_idx, (combined_ppg_batch, combined_ecg_batch, combined_seg_dbp_batch, combined_seg_sbp_batch) in enumerate(val_data_generator):
    
    output_file = os.path.join(output_dir, f'val_vg_images_batch_{batch_idx + 1}.npz')
    if os.path.exists(output_file):
        print(f"Batch {batch_idx + 1} already processed. Skipping...")
        continue
        
    print(f"Processing Batch {batch_idx + 1}...")

    with ThreadPoolExecutor() as executor:
        vg_images = list(executor.map(process_signal, range(len(combined_ppg_batch)), combined_ppg_batch, [vg_image_size]*len(combined_ppg_batch)))
        
    np.savez_compressed(output_file, vg_images=vg_images)
    print(f"Batch {batch_idx + 1} processing complete.")